In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os.path as osp
import sys


def add_path(path):
    if path not in sys.path:
        sys.path.insert(0, path)


this_dir = osp.dirname('../../train.py')

lib_path = osp.join(this_dir, 'lib')
add_path(lib_path)

In [2]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
import pandas as pd
import time

In [3]:
from torch.profiler import profile, record_function, ProfilerActivity


In [4]:
import torch
from torch import nn
from utils.dataloader import CERN_Dataset_V3, UMASS_Dataset_V2, DRED_Dataset
from torch.utils.data import DataLoader
from models.models import HAE, AE, SCSAE, HAE_V2
from models import gdn

#### CER

In [5]:
hae_48_cern = "../../model_param/state_7218799.pt"
scsae_48_cern = "../../model_param/state_6603495.pt"

hae_model = HAE_V2(latent_n = 48, input_days = 12, wide_freq = 48, reshape_factor = 1, device = 'cpu')
hae_model.load_state_dict(torch.load(hae_48_cern)['state_dict'])
hae_model.eval()
scsae_model = SCSAE(latent_n = 48, input_days = 12, wide_freq = 48, reshape_factor = 2)
scsae_model.load_state_dict(torch.load(scsae_48_cern)['state_dict'])
scsae_model.eval()
print('loaded')

loaded


In [6]:
hae_test_set = CERN_Dataset_V3(train = False, no_days = 12, reshape_factor = 1, mode = 'cnn')
scsae_test_set = CERN_Dataset_V3(train = False, no_days = 12, reshape_factor = 2, mode = 'cnn')
hae_loader = DataLoader(hae_test_set, batch_size = 1)
scsae_loader = DataLoader(scsae_test_set, batch_size = 1)

100%|██████████| 200/200 [00:00<00:00, 2890.34it/s]


SCSAE

In [16]:
times = []

for i in range(50):
    with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory = True) as prof:
        with record_function("model_inference"):
            scsae_model.encoder(scsae_test_set.cern_data[i].unsqueeze(0))
    times.append(prof.key_averages().total_average().self_cpu_time_total / 1000)

In [17]:
np.array(times).mean()

4.7964400000000005

In [18]:
np.array(times).std()

1.4566078286210051

In [19]:
print(prof.key_averages().table(row_limit=25))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::zeros         1.59%      38.000us         2.39%      57.000us      57.000us           4 b           0 b             1  
                      aten::empty         3.64%      87.000us         3.64%      87.000us       2.900us      50.39 Kb      50.39 Kb            30  
                      aten::zero_         0.17%       4.000us         0.17%       4.000us       4.000us           0 b           0 b             1  
                  model_inference        21.47%     513.000us        97.61%       2.332ms       2.332ms         

HAE

In [20]:
times = []

for i in range(50):
    with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory = True) as prof:
        with record_function("model_inference"):
            hae_model.encoder(hae_test_set.cern_data[i].unsqueeze(0))
    times.append(prof.key_averages().total_average().self_cpu_time_total / 1000)

In [21]:
np.array(times).mean()

9.71248

In [22]:
np.array(times).std()

0.7575105607184628

In [23]:
print(prof.key_averages().table(row_limit=25))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::zeros         1.14%     102.000us         1.38%     123.000us      41.000us         388 b           0 b             3  
                      aten::empty         2.49%     222.000us         2.49%     222.000us       1.337us     124.64 Kb     124.64 Kb           166  
                      aten::zero_         0.06%       5.000us         0.06%       5.000us       1.667us           0 b           0 b             3  
                  model_inference         9.46%     843.000us        98.81%       8.806ms       8.806ms         

#### UMASS

SCSAE

In [52]:
hae_96_umass = "../../model_param/state_3995498.pt"
scsae_96_umass = "../../model_param/state_5734868.pt"
hae_model = HAE_V2(latent_n = 96, input_days = 12, wide_freq = 96, reshape_factor = 1, device = 'cpu')
hae_model.load_state_dict(torch.load(hae_96_umass)['state_dict'])
hae_model.eval()
scsae_model = SCSAE(latent_n = 96, input_days = 12, wide_freq = 96, reshape_factor = 2)
scsae_model.load_state_dict(torch.load(scsae_96_umass)['state_dict'])
scsae_model.eval()
print('loaded')

loaded


In [53]:
hae_test_set = UMASS_Dataset_V2(train = False, no_days = 12, reshape_factor = 1, mode = 'cnn')
scsae_test_set = UMASS_Dataset_V2(train = False, no_days = 12, reshape_factor = 2, mode = 'cnn')
hae_loader = DataLoader(hae_test_set, batch_size = 1)
scsae_loader = DataLoader(scsae_test_set, batch_size = 1)

100%|██████████| 10/10 [00:00<00:00, 1639.94it/s]


In [54]:
times = []

for i in range(1):
    with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory = True) as prof:
        with record_function("model_inference"):
            scsae_model.encoder(scsae_test_set.umass_data[i].unsqueeze(0))
    times.append(prof.key_averages().total_average().self_cpu_time_total / 1000)

In [55]:
np.array(times).mean()

5.949

In [56]:
np.array(times).std()

0.0

In [59]:
print(prof.key_averages().table(row_limit=1000))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::zeros         0.42%      25.000us         0.74%      44.000us      44.000us           4 b           0 b             1  
                      aten::empty         1.51%      90.000us         1.51%      90.000us       3.000us      99.89 Kb      99.89 Kb            30  
                      aten::zero_         0.05%       3.000us         0.05%       3.000us       3.000us           0 b           0 b             1  
                  model_inference        16.39%     975.000us        99.26%       5.905ms       5.905ms         

HAE

In [60]:
times = []

for i in range(1):
    with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory = True) as prof:
        with record_function("model_inference"):
            hae_model.encoder(hae_test_set.umass_data[i].unsqueeze(0))
    times.append(prof.key_averages().total_average().self_cpu_time_total / 1000)

In [61]:
np.array(times).mean()

11.119

In [62]:
np.array(times).std()

0.0

In [63]:
print(prof.key_averages().table(row_limit=500))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::zeros         0.40%      44.000us         0.62%      69.000us      23.000us         388 b           0 b             3  
                      aten::empty         2.68%     298.000us         2.68%     298.000us       1.795us     124.64 Kb     124.64 Kb           166  
                      aten::zero_         0.04%       4.000us         0.04%       4.000us       1.333us           0 b           0 b             3  
                  model_inference        12.42%       1.381ms        99.65%      11.080ms      11.080ms         

#### DRED

SCSAE

In [64]:
hae_300_dred = "../../model_param/state_7437386.pt"
scsae_300_dred = "../../model_param/state_4128850.pt"
hae_model = HAE_V2(latent_n = 300, input_days = 60, wide_freq = 60, reshape_factor = 1, device = 'cpu')
hae_model.load_state_dict(torch.load(hae_300_dred)['state_dict'])
hae_model.eval()
scsae_model = SCSAE(latent_n = 300, input_days = 60, wide_freq = 60, reshape_factor = 1)
scsae_model.load_state_dict(torch.load(scsae_300_dred)['state_dict'])
scsae_model.eval()
print('loaded')

loaded


In [65]:
hae_test_set = DRED_Dataset(train = False, no_rows = 60, reshape_factor = 1, mode = 'cnn')
scsae_test_set = DRED_Dataset(train = False, no_rows = 60, reshape_factor = 1, mode = 'cnn')
hae_loader = DataLoader(hae_test_set, batch_size = 1)
scsae_loader = DataLoader(scsae_test_set, batch_size = 1)

DRED processed.
DRED processed.


In [67]:
times = []

for i in range(1):
    with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory = True) as prof:
        with record_function("model_inference"):
            scsae_model.encoder(scsae_test_set.dred_data[i].unsqueeze(0))
    times.append(prof.key_averages().total_average().self_cpu_time_total / 1000)

In [68]:
np.array(times).mean()

3.194

In [69]:
np.array(times).std()

0.0

In [71]:
print(prof.key_averages().table(row_limit=500))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::zeros         0.72%      23.000us         1.25%      40.000us      40.000us           4 b           0 b             1  
                      aten::empty         3.85%     123.000us         3.85%     123.000us       4.100us     310.27 Kb     310.27 Kb            30  
                      aten::zero_         0.09%       3.000us         0.09%       3.000us       3.000us           0 b           0 b             1  
                  model_inference        20.29%     648.000us        98.75%       3.154ms       3.154ms         

HAE

In [72]:
times = []

for i in range(1):
    with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory = True) as prof:
        with record_function("model_inference"):
            hae_model.encoder(hae_test_set.dred_data[i].unsqueeze(0))
    times.append(prof.key_averages().total_average().self_cpu_time_total / 1000)

In [73]:
np.array(times).mean()

40.663

In [74]:
np.array(times).std()

0.0

In [76]:
print(prof.key_averages().table(row_limit=5000))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::zeros         0.14%      58.000us         0.22%      91.000us      30.333us         388 b           0 b             3  
                      aten::empty         3.01%       1.224ms         3.01%       1.224ms       1.895us     295.11 Kb     295.11 Kb           646  
                      aten::zero_         0.02%       7.000us         0.02%       7.000us       2.333us           0 b           0 b             3  
                  model_inference         8.64%       3.513ms        99.86%      40.606ms      40.606ms         